In [1]:
%load_ext autoreload
%autoreload 2

import sys
import lightning.pytorch as pl
import torch
from matplotlib import pyplot as plt
import numpy as np
import torch
from torch import nn
import os

sys.path.insert(1, sys.path[0] + '/..')
from src.data.datamodule import DataModule
from src.model.setup import setup_model
from src.misc.utils import set_seed_and_precision
from src.misc.files import read_image
from src.data.utils import pad_tensor, pad_list

from src.run import parse_option
args = parse_option(notebook=True)

ImportError: cannot import name 'pad' from 'src.data.utils' (/Users/elias/Documenten MBA/Studie/AIMedicalImaging/notebooks/../src/data/utils.py)

In [5]:
x_path = '../data_dev/boxes/train/images/'
y_path = '../data_dev/boxes/train/labels/'
for file in os.listdir(x_path):
    patches = os.listdir(os.path.join(x_path, file))
    for patch in patches:
        boxes = os.listdir(os.path.join(x_path, file, patch))
        for box in boxes:
            x = np.load(os.path.join(x_path, file, patch, box))
            y = np.load(os.path.join(y_path, file, patch, box))
            print(x.shape)


(20, 34, 18)
(21, 36, 19)
(20, 37, 21)
(21, 29, 19)
(12, 15, 9)
(26, 24, 20)
(2, 11, 5)
(14, 16, 4)
(15, 16, 10)
(16, 29, 17)
(21, 23, 14)
(20, 36, 12)


# Dev single image

In [3]:
image_data, _ = read_image('../data_dev/train/images/RibFrac422-image.nii.gz') 
label_data, header = read_image('../data_dev/train/labels/RibFrac422-label.nii.gz')

x = image_data
x = (x - x.min()) / (x.max() - x.min())

y = label_data
y = np.where(y > 0, 1, 0)

In [4]:
# coords = np.where(y == 1)

# for coord, coord_name in zip(coords, ['x', 'y', 'z']):
#     plt.hist(coord, bins=50, label = coord_name, alpha = 0.5)
# plt.legend()
# plt.show()

In [5]:
coords = [
    [[300, 420], [200, 250], [30, 111]],
    [[400, 500], [150, 250], [100, 201]],
]

def coord_slice(x, coord):
    c = tuple([slice(*coord_xyz) for coord_xyz in coord])
    return x[c]

for coord in coords:
    y_slice = coord_slice(y, coord)
    print(y_slice.shape, y_slice.sum())


x0 = coord_slice(x, coords[0])
y0 = coord_slice(y, coords[0])
x1 = coord_slice(x, coords[1])
y1 = coord_slice(y, coords[1])

x0 = torch.from_numpy(x0).float()
y0 = torch.from_numpy(y0).float()
x1 = torch.from_numpy(x1).float()
y1 = torch.from_numpy(y1).float()

x_unpadded = [x0, x1]
y_unpadded = [y0, y1]

(120, 50, 81) 1387
(100, 100, 101) 16372


In [6]:
def pad(x_unpadded, padding_value = 0, size = 64):
    return torch.stack([nn.functional.pad(x, (
            0, size - x.shape[-1],
            0, size - x.shape[-2], 
            0, size - x.shape[-3]
        ), value = padding_value) for x in x_unpadded])

x_padded = pad(x_unpadded)
y_padded = pad(y_unpadded)

for split in ['train', 'val', 'test']:
    for i, x_i in enumerate(x_padded):
        torch.save(x_i, f'../data_boxes/{split}/images/{i}.pt')

    for i, y_i in enumerate(y_padded):
        torch.save(y_i, f'../data_boxes/{split}/labels/{i}.pt')